# Import Statements

In [ ]:
%%capture

!pip install --upgrade scikit-learn

In [ ]:
import numpy as np
import pandas as pd
import scikitplot
import matplotlib.pyplot as plt
from scipy import stats

from tensorflow import keras
from sklearn.metrics import classification_report, top_k_accuracy_score

Since this competition is closed, we have access to the full dataset. Therefore, we will combine the private and public test data sets together. 

# Read in Full Data Set

In [ ]:
# Read in full data set
data = pd.read_csv('../input/challenges-in-representation-learning-facial-expression-recognition-challenge/icml_face_data.csv')
data.columns = ['emotion', 'Usage', 'pixels']
print(data.shape)

In [ ]:
# View first five rows
data.head()

# Select Only Data in Test Sets

In [ ]:
# Select only rows that are in the public or private test set
test = data.loc[data["Usage"] != 'Training',['emotion','pixels']]
#test.drop(columns='Usage', inplace=True)
test.head()

# Reshape Pixels

In [ ]:
# Reshape the pixels
test['pixels'] = [np.fromstring(x, dtype=int, sep=' ').reshape(-1,48,48,1) for x in test['pixels']]

In [ ]:
# Combine pixels into single array
pixels = np.concatenate(test['pixels'].values)

print(pixels.shape)

In [ ]:
# Standardize the pixels values between 0 and 1
pixels = pixels / 255

# Load Trained Model

In [ ]:
# Load model
model = keras.models.load_model('../input/models/Facial Recognition Models/fer_v05_BZ.h5')

# Generate Prediction Probabilities

In [ ]:
# Compute probabilities
test_probs = model.predict(pixels)

# Assign Each Sample a Predicted Label

In [ ]:
final_pred = np.argmax(test_probs, axis=1)

# Combine Predicted Labels and Actual Labels

In [ ]:
test['predictions'] = np.round(final_pred,0)
test.head()

In [ ]:
emotion_cat = {0:'Anger', 1:'Disgust', 2:'Fear', 3:'Happiness', 4: 'Sadness', 5: 'Surprise', 6: 'Neutral'}
test['emotion'] = test['emotion'].apply(lambda x: emotion_cat[x])
test['predictions'] = test['predictions'].apply(lambda x: emotion_cat[x])

# Classification Report

In [ ]:
my_classification_report = classification_report(test['emotion'], test['predictions'])
print(my_classification_report)

Our CNN model using the Xception architecure had a 66% accuracy. Since the competition has closed, we cannot submit our predictions or be placed on the leaderboard. Based on the submissions while the competition was still active, we would have placed in the top five teams. The model was able to predict images with happiness better than the other emotions, and struggled with classifying fear and disgust. For example, disgust had a precision of 61% and a recall of 38%. This means of the images our model classified the emotion as disgust, 61% of these images true emotion was digust. However, the model correctly indentified only 38% of all of the images with disgust as the emotion. Below, we will look at the confustion matrix to see what emotions our model thought these images were.

# Number of Misclassified Samples

In [ ]:
print('Total Wrong Predictions:', np.sum(test['emotion'] != test['predictions']))

# Confusion Matrix

In [ ]:
scikitplot.metrics.plot_confusion_matrix(test['emotion'], test['predictions'], figsize=(7,7))    

The confustion matrix allows us to see a breakdown of the predicted labels and the true labels. As mentioned previously, our model had a hard time classifying images with disgust. We can see that our model correctly predicted 42 images where the true emotion was disgust. However, it predicted that the emotion was anger when it was actually disgust 43 times. In the future, we may decide to combine the anger and digust classes since they are very similar emotions. We also see that our model had a tough time distinguishing between neutral and sadness.

# Top-K Accuracy

In [ ]:
# Compute Top-K accuracy for each class
for k in range(0, 7):
    print(f"{emotion_cat[k]} top accuracy: {round(top_k_accuracy_score(test['emotion'], test_probs, k=k), 2)}")

# View Correctly and Incorrectly Classified Samples

## Correctly Classified

Below is a plot of correctly classified images. The blue text next to each image is the observed emotion and the red text is the predicted emotion.

In [ ]:
plt.close()
plt.rcParams["figure.figsize"] = [16,16]

row = 0
for emotion in np.unique(test['emotion'].values):
    all_emotion_images = test[(test['emotion'] == emotion) & (test['predictions'] == emotion)]
    for i in range(5):
        
        img = all_emotion_images.iloc[i,].pixels.reshape(48,48)
        actual_lab = emotion
        predicted_lab = all_emotion_images.iloc[i,].predictions

        plt.subplot(7,5,row+i+1)
        plt.imshow(img, cmap='binary_r')
        plt.text(-30, 5, s = str(actual_lab), fontsize=10, color='b')
        plt.text(-30, 10, s = str(predicted_lab), fontsize=10, color='r')
        plt.axis('off')
    row += 5
    
plt.show()

## Incorrectly Classified

Below is a plot of misclassified images. The blue text next to each image is the observed emotion and the red text is the predicted emotion.

In [ ]:
plt.close()
plt.rcParams["figure.figsize"] = [16,16]

row = 0
for emotion in np.unique(test['emotion'].values):
    all_emotion_images = test[(test['emotion'] == emotion) & (test['predictions'] != emotion)]
    for i in range(5):
        
        img = all_emotion_images.iloc[i,].pixels.reshape(48,48)
        actual_lab = emotion
        predicted_lab = all_emotion_images.iloc[i,].predictions

        plt.subplot(7,5,row+i+1)
        plt.imshow(img, cmap='binary_r')
        plt.text(-30, 5, s = str(actual_lab), fontsize=10, color='b')
        plt.text(-30, 10, s = str(predicted_lab), fontsize=10, color='r')
        plt.axis('off')
    row += 5
    
plt.show()